In [8]:
import sys
sys.path.append('..')

In [9]:
import questpipe as qp

# Define and run pipeline

In [10]:
arguments = qm.Arguments(
    msub_arguments=[
        "-A b1042",
        "-q pulrseq",
        "-l walltime=24:00:00,nodes=1:ppn=8",
        "-m a",
        "-j oe",
        "-W umask=0113",
        "-N {job_name}"],
    workdir="/home/htn551/",
    outdir="/home/htn551/",
    errdir="/home/htn551/")

In [11]:
ssh = qp.SecureShell("quest.northwestern.edu", "htn551", "../ssh_keys/quest_rsa")
pipeline = qp.Pipeline(
    name="mypipeline", 
    shell=ssh, 
    arguments=arguments,
    debug_filename="pipeline.log",    # local file, not in quest ; error if file exists
    state_filename="pipeline.json")   # local file, not in quest ; error if file exists
)

with pipeline:
    t1 = pipeline.create_job(name="task1")
    t1.prepare_async_run("""
        sleep 5
        echo {job_name} > test_file
        """)

    t2 = pipeline.create_job(name="task2", dependences=[t1])
    t2.async_run("""
        echo {job_name} >> test_file
        """)

    t3 = pipeline.create_job(name="task3", dependences=[t1])
    t3.async_run("""
        echo {job_name} >> test_file
        """)

    t4 = pipeline.create_job(name="task4", dependences=[t2, t3])
    t4.async_run("""
        sleep 1
        echo {job_name} >> test_file
        """)

    t1.unhold()

AttributeError: __exit__

# Check pipeline status

In [ ]:
ssh = qp.SecureShell("quest.northwestern.edu", "htn551", "../ssh_keys/quest_rsa")
pipeline = qp.Pipeline.from_json("pipeline.json", shell=ssh)
pipeline.show_status()

# Crazy ideas

In [ ]:
pipeline.run_batch("""
task t1:
    sleep 5
    echo {job_name} > test_file
    
task t2: t1
    echo {job_name} >> test_file
    
task t3: t1
    echo {job_name} >> test_file

task t4: t2, t3
    sleep 1
    echo {job_name} >> test_file
""")